In [1]:
import sys

import tensorflow as tf
from tensorflow import keras as kr

from time import sleep
import numpy as np
import random
import pandas as pd
pd.set_option('display.max_rows', 200)

import sklearn as sk

from sklearn.preprocessing import MinMaxScaler

import matplotlib 

import matplotlib.pyplot as plt

import zipfile
import os

import supportFunctions as sf

import inspect
lines = inspect.getsource(sf.highest)

In [2]:
#função utilizada para normalizar os dados de treino e validação
def normalize_data(train, test):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(train)
    train_scaled = scaler.transform(train)
    test_scaled = scaler.transform(test)
    
    return scaler, train_scaled, test_scaled

#função utilizada para desnormalizar os dados, no formato de entrada
def denormalize_data(scaler,data):
    result = scaler.inverse_transform(data)
    return result

#função utilizada para desnomalizar o valor predito
def denormalize_prediction(scaler, dataX, dataY):
    formatted_data = np.array(dataX)
    formatted_data[:,3] = np.array(dataY)
    
    return denormalize_data(scaler,formatted_data)[:,3]

In [3]:
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")
# tf.debugging.set_log_device_placement(False)

Tensor Flow Version: 2.12.0
Keras Version: 2.12.0

Python 3.9.16 (main, Jan 11 2023, 16:16:36) [MSC v.1916 64 bit (AMD64)]
Pandas 1.5.2
Scikit-Learn 1.2.1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is NOT AVAILABLE


In [4]:
#quantos passos para trás os indicadores técnicos vão olhar
ti_memory = 10

#lendo os dados que serão utilizados para treinamento e validação
ge1day = pd.read_csv('Data\GE.csv')
#removendo a coluna que indica a data
ge1day = ge1day.drop(['Date'], axis = 1)

df = ge1day
#reseting the index
df.index = range(len(df))

In [7]:
#adicionando os parametros de indicadores tecnicos ao dataframe
df["""highest_{}""".format(ti_memory)] = sf.highest(df.Close,ti_memory)
df["""lowest_{}""".format(ti_memory)] = sf.lowest(df.Close,ti_memory)
df["""wma_{}""".format(ti_memory)] = sf.wma(df.Close,ti_memory)
df["""ema_{}""".format(ti_memory)] = sf.ema(df.Close,ti_memory)
df["""hma_{}""".format(ti_memory)] = sf.hma(df.Close,ti_memory)
df["""macd_12_26"""] = sf.macd(df.Close,12,26)
df["""rsi_{}""".format(ti_memory)] = sf.rsi(df.Close,ti_memory)
df["""dpo_{}""".format(ti_memory)] = sf.dpo(df.Close,ti_memory)

#parametros nao utilizados:
# df["""sma_{}""".format(ti_memory)] = sf.sma(df.Close,ti_memory)
# df["""so_k_5"""] = sf.so_k(df.Close)
# df["""so_d_3"""] = sf.so_d(df.Close)
# df["""obv""".format(ti_memory)] = sf.obv(df.Close,df.Volume)

C:\Users\guilh\PythonCodes\supportFunctions.py:263: RuntimeWarning: divide by zero encountered in divide
  rs = ag_vector/al_vector #numpy arrays so therefore can use element wise division
C:\Users\guilh\PythonCodes\supportFunctions.py:263: RuntimeWarning: invalid value encountered in divide
  rs = ag_vector/al_vector #numpy arrays so therefore can use element wise division


In [8]:
display(df)

,Open,High,Low,Close,Adj Close,Volume,highest_10,lowest_10,wma_10,ema_10,hma_10,macd_12_26,rsi_10,dpo_10
0,0.751202,0.763722,0.743690,0.748698,0.001782,2156500,0.748698,0.748698,0.748698,0.748698,0.748698,0.000000,NaN,0.000000
1,0.744942,0.744942,0.738682,0.741186,0.001764,1477600,0.748698,0.741186,0.743690,0.744942,0.745359,0.000000,100.000000,-0.744942
2,0.741186,0.747446,0.726162,0.732422,0.001743,1837000,0.748698,0.732422,0.738056,0.740769,0.741708,0.000000,100.000000,-0.740769
3,0.732422,0.733674,0.701122,0.713642,0.001698,2725600,0.748698,0.713642,0.728290,0.733987,0.734112,0.000000,100.000000,-0.733987
4,0.713642,0.713642,0.691106,0.712390,0.001695,3095000,0.748698,0.712390,0.722990,0.729667,0.727268,0.000000,100.000000,-0.729667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14658,7.630000,8.300000,7.510000,8.120000,8.120000,123180900,8.120000,6.110000,7.062182,7.424700,7.239323,-1.116727,50.108961,-0.399000
14659,7.680000,7.870000,7.540000,7.620000,7.620000,93299000,8.120000,6.110000,7.175091,7.460209,7.697586,-1.029444,40.027642,-0.496000
14660,7.540000,7.940000,7.350000,7.890000,7.890000,86850200,8.120000,6.110000,7.341273,7.538353,8.019525,-0.927790,41.907705,-0.579000
14661,7.870000,8.180000,7.820000,7.940000,7.940000,121149900,8.120000,6.110000,7.494182,7.611380,8.165303,-0.833585,32.316173,-1.075000


In [9]:
#jogando alguns dados fora para termos todas variaveis incializadas
inicialization_steps = 30
prediction_ahead = 1

dataValues = df.values

trainSize = int(len(dataValues)*0.8)
testSize = len(dataValues) - trainSize

train = df.head(trainSize)
# train,minDataTrain,minMaxDataTrain = sf.normalizaTrain(train)
test = df.tail(testSize)
# test = sf.normalizaTest(test, minDataTrain, minMaxDataTrain)

scaler, train_normalized, test_normalized = normalize_data(train, test)

#deixando um valor de fora, para podermos prever o próximo valor quando for o último:
trainX = train_normalized[inicialization_steps:(len(train)-prediction_ahead)]
testX = test_normalized[0:(len(test)-prediction_ahead)]

#normalizando as entradas
# scalerX, trainX, testX = normalize_data(trainX, testX)

#pegando apenas o valor que queremos prever
trainY = train_normalized[(inicialization_steps+prediction_ahead):len(train),3]
testY = test_normalized[prediction_ahead:len(test),3]

#calculando a diferenca percentual do preco de fechamento entre um dia e outro
# trainY = (train.values[(inicialization_steps+1):len(train),3]/trainX[:,3])-1
# testY = (test.values[1:len(test),3]/testX[:,3])-1


# scalerY, trainY, testY = normalize_data(trainY, testY)

In [10]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(11699, 14)
(11699,)
(2932, 14)
(2932,)


In [11]:
#checking if it makes sense
# print(trainInputX[:10,(rnn_memory-1),3])
# print(trainInputY[:10])

In [12]:
rnn_memory = 50

trainInputX, trainInputY = sf.prepareData(trainX, trainY, rnn_memory)
testInputX, testInputY = sf.prepareData(testX, testY, rnn_memory)
print(trainInputX.shape)
print(trainInputY.shape)
print(testInputX.shape)
print(testInputY.shape)

#validacao tamanho apos tratamentos
print(len(trainX))
print(len(trainInputX))

print(len(trainY))
print(len(trainInputY))

trainInputX = trainInputX [:,:,0:14]
testInputX = testInputX [:,:,0:14]
print(trainInputX.shape)

trained_models_rnn_50 = []
trained_models_rnn_50_history = []

for i in range(0,20):

    #giving it reproducibility
    seed = (i+1000)

    os.environ['PYTHONHASHseed']=str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    batch_size = 256

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
    
    trained_models_rnn_50.append(kr.Sequential())
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_50[i].add(kr.layers.SimpleRNN(14,return_sequences = True))
    trained_models_rnn_50[i].add(kr.layers.SimpleRNN(8))
    # rnn_model.add(kr.layers.SimpleRNN(5,input_shape=(trainInputX.shape[1],trainInputX.shape[2])))
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_50[i].add(kr.layers.Dense(1))
    trained_models_rnn_50[i].compile(loss='mean_squared_error',optimizer='adam')
    
    trained_models_rnn_50_history.append(trained_models_rnn_50[i].fit(trainInputX, trainInputY, epochs=2000, batch_size=batch_size, verbose = 1, validation_data=(testInputX,testInputY), callbacks=[callback]))
    
    trained_models_rnn_50[i].save('C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_50_batch_256_earlystop_valloss_'+str(i))

(11649, 50, 14)
(11649,)
(2882, 50, 14)
(2882,)
11699
11649
11699
11649
(11649, 50, 14)
Epoch 1/2000
46/46 [==============================] - 2s 12ms/step - loss: 0.0345 - val_loss: 0.0254
Epoch 2/2000
46/46 [==============================] - 0s 8ms/step - loss: 0.0053 - val_loss: 0.0162
Epoch 3/2000
46/46 [==============================] - 0s 8ms/step - loss: 0.0026 - val_loss: 0.0106
Epoch 4/2000
46/46 [==============================] - 0s 8ms/step - loss: 0.0017 - val_loss: 0.0071
Epoch 5/2000
46/46 [==============================] - 0s 8ms/step - loss: 0.0013 - val_loss: 0.0049
Epoch 6/2000
46/46 [==============================] - 0s 8ms/step - loss: 9.9089e-04 - val_loss: 0.0037
Epoch 7/2000
46/46 [==============================] - 0s 8ms/step - loss: 8.1820e-04 - val_loss: 0.0030
Epoch 8/2000
46/46 [==============================] - 0s 8ms/step - loss: 6.9838e-04 - val_loss: 0.0028
Epoch 9/2000
46/46 [==============================] - 0s 8ms/step - loss: 6.0885e-04 - val_loss: 0.

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_50_batch_256_earlystop_valloss_0\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_50_batch_256_earlystop_valloss_0\assets


Epoch 1/2000
46/46 [==============================] - 2s 12ms/step - loss: 0.0224 - val_loss: 0.0119
Epoch 2/2000
46/46 [==============================] - 0s 8ms/step - loss: 0.0040 - val_loss: 0.0061
Epoch 3/2000
46/46 [==============================] - 0s 8ms/step - loss: 0.0024 - val_loss: 0.0038
Epoch 4/2000
46/46 [==============================] - 0s 8ms/step - loss: 0.0018 - val_loss: 0.0026
Epoch 5/2000
46/46 [==============================] - 0s 8ms/step - loss: 0.0013 - val_loss: 0.0020
Epoch 6/2000
46/46 [==============================] - 0s 8ms/step - loss: 0.0010 - val_loss: 0.0014
Epoch 7/2000
46/46 [==============================] - 0s 8ms/step - loss: 7.6511e-04 - val_loss: 0.0013
Epoch 8/2000
46/46 [==============================] - 0s 8ms/step - loss: 5.9717e-04 - val_loss: 0.0012
Epoch 9/2000
46/46 [==============================] - 0s 8ms/step - loss: 4.8444e-04 - val_loss: 0.0014
Epoch 10/2000
46/46 [==============================] - 0s 8ms/step - loss: 4.0761e-04 -

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_50_batch_256_earlystop_valloss_1\assets


...

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_50_batch_256_earlystop_valloss_18\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_50_batch_256_earlystop_valloss_18\assets


Epoch 1/2000
46/46 [==============================] - 2s 11ms/step - loss: 0.0307 - val_loss: 0.0104
Epoch 2/2000
46/46 [==============================] - 0s 8ms/step - loss: 0.0018 - val_loss: 0.0077
Epoch 3/2000
46/46 [==============================] - 0s 8ms/step - loss: 0.0011 - val_loss: 0.0056
Epoch 4/2000
46/46 [==============================] - 0s 7ms/step - loss: 7.7044e-04 - val_loss: 0.0043
Epoch 5/2000
46/46 [==============================] - 0s 8ms/step - loss: 5.5618e-04 - val_loss: 0.0034
Epoch 6/2000
46/46 [==============================] - 0s 8ms/step - loss: 4.2788e-04 - val_loss: 0.0029
Epoch 7/2000
46/46 [==============================] - 0s 8ms/step - loss: 3.5291e-04 - val_loss: 0.0023
Epoch 8/2000
46/46 [==============================] - 0s 7ms/step - loss: 3.0859e-04 - val_loss: 0.0023
Epoch 9/2000
46/46 [==============================] - 0s 7ms/step - loss: 2.7657e-04 - val_loss: 0.0023
Epoch 10/2000
46/46 [==============================] - 0s 8ms/step - loss: 

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_50_batch_256_earlystop_valloss_19\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_50_batch_256_earlystop_valloss_19\assets


In [13]:
rnn_memory = 100

trainInputX, trainInputY = sf.prepareData(trainX, trainY, rnn_memory)
testInputX, testInputY = sf.prepareData(testX, testY, rnn_memory)
print(trainInputX.shape)
print(trainInputY.shape)
print(testInputX.shape)
print(testInputY.shape)

#validacao tamanho apos tratamentos
print(len(trainX))
print(len(trainInputX))

print(len(trainY))
print(len(trainInputY))

trainInputX = trainInputX [:,:,0:14]
testInputX = testInputX [:,:,0:14]
print(trainInputX.shape)

trained_models_rnn_100 = []
trained_models_rnn_100_history = []

for i in range(0,20):

    #giving it reproducibility
    seed = (i+1000)

    os.environ['PYTHONHASHseed']=str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    batch_size = 256

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
    
    trained_models_rnn_100.append(kr.Sequential())
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_100[i].add(kr.layers.SimpleRNN(14,return_sequences = True))
    trained_models_rnn_100[i].add(kr.layers.SimpleRNN(8))
    # rnn_model.add(kr.layers.SimpleRNN(5,input_shape=(trainInputX.shape[1],trainInputX.shape[2])))
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_100[i].add(kr.layers.Dense(1))
    trained_models_rnn_100[i].compile(loss='mean_squared_error',optimizer='adam')
    
    trained_models_rnn_100_history.append(trained_models_rnn_100[i].fit(trainInputX, trainInputY, epochs=2000, batch_size=batch_size, verbose = 1, validation_data=(testInputX,testInputY), callbacks=[callback]))
    
    trained_models_rnn_100[i].save('C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_100_batch_256_earlystop_valloss_'+str(i))

(11599, 100, 14)
(11599,)
(2832, 100, 14)
(2832,)
11699
11599
11699
11599
(11599, 100, 14)
Epoch 1/2000
46/46 [==============================] - 2s 20ms/step - loss: 0.0339 - val_loss: 0.0212
Epoch 2/2000
46/46 [==============================] - 1s 16ms/step - loss: 0.0050 - val_loss: 0.0091
Epoch 3/2000
46/46 [==============================] - 1s 16ms/step - loss: 0.0020 - val_loss: 0.0045
Epoch 4/2000
46/46 [==============================] - 1s 15ms/step - loss: 0.0012 - val_loss: 0.0032
Epoch 5/2000
46/46 [==============================] - 1s 16ms/step - loss: 8.8122e-04 - val_loss: 0.0028
Epoch 6/2000
46/46 [==============================] - 1s 16ms/step - loss: 7.4692e-04 - val_loss: 0.0027
Epoch 7/2000
46/46 [==============================] - 1s 16ms/step - loss: 6.5854e-04 - val_loss: 0.0024
Epoch 8/2000
46/46 [==============================] - 1s 16ms/step - loss: 5.9091e-04 - val_loss: 0.0023
Epoch 9/2000
46/46 [==============================] - 1s 16ms/step - loss: 5.3316e-04

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_100_batch_256_earlystop_valloss_0\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_100_batch_256_earlystop_valloss_0\assets


Epoch 1/2000
46/46 [==============================] - 2s 20ms/step - loss: 0.0224 - val_loss: 0.0120
Epoch 2/2000
46/46 [==============================] - 1s 16ms/step - loss: 0.0039 - val_loss: 0.0056
Epoch 3/2000
46/46 [==============================] - 1s 15ms/step - loss: 0.0024 - val_loss: 0.0034
Epoch 4/2000
46/46 [==============================] - 1s 16ms/step - loss: 0.0017 - val_loss: 0.0023
Epoch 5/2000
46/46 [==============================] - 1s 15ms/step - loss: 0.0013 - val_loss: 0.0016
Epoch 6/2000
46/46 [==============================] - 1s 15ms/step - loss: 9.7418e-04 - val_loss: 0.0013
Epoch 7/2000
46/46 [==============================] - 1s 15ms/step - loss: 7.4212e-04 - val_loss: 0.0012
Epoch 8/2000
46/46 [==============================] - 1s 15ms/step - loss: 5.7746e-04 - val_loss: 0.0012
Epoch 9/2000
46/46 [==============================] - 1s 15ms/step - loss: 4.6824e-04 - val_loss: 0.0012
Epoch 10/2000
46/46 [==============================] - 1s 16ms/step - loss:

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_100_batch_256_earlystop_valloss_1\assets


...

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_100_batch_256_earlystop_valloss_18\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_100_batch_256_earlystop_valloss_18\assets


Epoch 1/2000
46/46 [==============================] - 2s 20ms/step - loss: 0.0319 - val_loss: 0.0104
Epoch 2/2000
46/46 [==============================] - 1s 16ms/step - loss: 0.0018 - val_loss: 0.0076
Epoch 3/2000
46/46 [==============================] - 1s 16ms/step - loss: 0.0011 - val_loss: 0.0059
Epoch 4/2000
46/46 [==============================] - 1s 16ms/step - loss: 7.7545e-04 - val_loss: 0.0043
Epoch 5/2000
46/46 [==============================] - 1s 16ms/step - loss: 5.6668e-04 - val_loss: 0.0032
Epoch 6/2000
46/46 [==============================] - 1s 16ms/step - loss: 4.3678e-04 - val_loss: 0.0027
Epoch 7/2000
46/46 [==============================] - 1s 17ms/step - loss: 3.6120e-04 - val_loss: 0.0024
Epoch 8/2000
46/46 [==============================] - 1s 17ms/step - loss: 3.1387e-04 - val_loss: 0.0023
Epoch 9/2000
46/46 [==============================] - 1s 16ms/step - loss: 2.8118e-04 - val_loss: 0.0020
Epoch 10/2000
46/46 [==============================] - 1s 16ms/step

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_100_batch_256_earlystop_valloss_19\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_100_batch_256_earlystop_valloss_19\assets


In [14]:
rnn_memory = 200

trainInputX, trainInputY = sf.prepareData(trainX, trainY, rnn_memory)
testInputX, testInputY = sf.prepareData(testX, testY, rnn_memory)
print(trainInputX.shape)
print(trainInputY.shape)
print(testInputX.shape)
print(testInputY.shape)

#validacao tamanho apos tratamentos
print(len(trainX))
print(len(trainInputX))

print(len(trainY))
print(len(trainInputY))

trainInputX = trainInputX [:,:,0:14]
testInputX = testInputX [:,:,0:14]
print(trainInputX.shape)

trained_models_rnn_200 = []
trained_models_rnn_200_history = []

for i in range(0,20):

    #giving it reproducibility
    seed = (i+1000)

    os.environ['PYTHONHASHseed']=str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    batch_size = 256

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
    
    trained_models_rnn_200.append(kr.Sequential())
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_200[i].add(kr.layers.SimpleRNN(14,return_sequences = True))
    trained_models_rnn_200[i].add(kr.layers.SimpleRNN(8))
    # rnn_model.add(kr.layers.SimpleRNN(5,input_shape=(trainInputX.shape[1],trainInputX.shape[2])))
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_200[i].add(kr.layers.Dense(1))
    trained_models_rnn_200[i].compile(loss='mean_squared_error',optimizer='adam')
    
    trained_models_rnn_200_history.append(trained_models_rnn_200[i].fit(trainInputX, trainInputY, epochs=2000, batch_size=batch_size, verbose = 1, validation_data=(testInputX,testInputY), callbacks=[callback]))
    
    trained_models_rnn_200[i].save('C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_200_batch_256_earlystop_valloss_'+str(i))

(11499, 200, 14)
(11499,)
(2732, 200, 14)
(2732,)
11699
11499
11699
11499
(11499, 200, 14)
Epoch 1/2000
45/45 [==============================] - 3s 35ms/step - loss: 0.0344 - val_loss: 0.0210
Epoch 2/2000
45/45 [==============================] - 1s 31ms/step - loss: 0.0052 - val_loss: 0.0091
Epoch 3/2000
45/45 [==============================] - 1s 31ms/step - loss: 0.0022 - val_loss: 0.0047
Epoch 4/2000
45/45 [==============================] - 1s 31ms/step - loss: 0.0013 - val_loss: 0.0033
Epoch 5/2000
45/45 [==============================] - 1s 32ms/step - loss: 9.2236e-04 - val_loss: 0.0028
Epoch 6/2000
45/45 [==============================] - 1s 32ms/step - loss: 7.7258e-04 - val_loss: 0.0028
Epoch 7/2000
45/45 [==============================] - 1s 32ms/step - loss: 6.7827e-04 - val_loss: 0.0024
Epoch 8/2000
45/45 [==============================] - 1s 31ms/step - loss: 6.0752e-04 - val_loss: 0.0024
Epoch 9/2000
45/45 [==============================] - 1s 32ms/step - loss: 5.4819e-04

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_200_batch_256_earlystop_valloss_0\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_200_batch_256_earlystop_valloss_0\assets


Epoch 1/2000
45/45 [==============================] - 3s 35ms/step - loss: 0.0227 - val_loss: 0.0122
Epoch 2/2000
45/45 [==============================] - 1s 31ms/step - loss: 0.0041 - val_loss: 0.0055
Epoch 3/2000
45/45 [==============================] - 1s 32ms/step - loss: 0.0025 - val_loss: 0.0035
Epoch 4/2000
45/45 [==============================] - 1s 32ms/step - loss: 0.0018 - val_loss: 0.0020
Epoch 5/2000
45/45 [==============================] - 1s 32ms/step - loss: 0.0014 - val_loss: 0.0016
Epoch 6/2000
45/45 [==============================] - 1s 32ms/step - loss: 0.0010 - val_loss: 0.0012
Epoch 7/2000
45/45 [==============================] - 1s 32ms/step - loss: 8.0013e-04 - val_loss: 0.0013
Epoch 8/2000
45/45 [==============================] - 1s 32ms/step - loss: 6.3825e-04 - val_loss: 0.0013
Epoch 9/2000
45/45 [==============================] - 1s 32ms/step - loss: 5.2576e-04 - val_loss: 0.0015
Epoch 10/2000
45/45 [==============================] - 1s 32ms/step - loss: 4.4

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_200_batch_256_earlystop_valloss_1\assets


...

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_200_batch_256_earlystop_valloss_18\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_200_batch_256_earlystop_valloss_18\assets


Epoch 1/2000
45/45 [==============================] - 3s 36ms/step - loss: 0.0359 - val_loss: 0.0127
Epoch 2/2000
45/45 [==============================] - 1s 33ms/step - loss: 0.0019 - val_loss: 0.0087
Epoch 3/2000
45/45 [==============================] - 1s 32ms/step - loss: 0.0012 - val_loss: 0.0066
Epoch 4/2000
45/45 [==============================] - 1s 33ms/step - loss: 8.7232e-04 - val_loss: 0.0051
Epoch 5/2000
45/45 [==============================] - 1s 32ms/step - loss: 6.3752e-04 - val_loss: 0.0039
Epoch 6/2000
45/45 [==============================] - 1s 31ms/step - loss: 4.8927e-04 - val_loss: 0.0033
Epoch 7/2000
45/45 [==============================] - 1s 33ms/step - loss: 4.0397e-04 - val_loss: 0.0030
Epoch 8/2000
45/45 [==============================] - 1s 32ms/step - loss: 3.5091e-04 - val_loss: 0.0026
Epoch 9/2000
45/45 [==============================] - 1s 33ms/step - loss: 3.1334e-04 - val_loss: 0.0025
Epoch 10/2000
45/45 [==============================] - 1s 32ms/step

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_200_batch_256_earlystop_valloss_19\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_200_batch_256_earlystop_valloss_19\assets


In [15]:
rnn_memory = 400

trainInputX, trainInputY = sf.prepareData(trainX, trainY, rnn_memory)
testInputX, testInputY = sf.prepareData(testX, testY, rnn_memory)
print(trainInputX.shape)
print(trainInputY.shape)
print(testInputX.shape)
print(testInputY.shape)

#validacao tamanho apos tratamentos
print(len(trainX))
print(len(trainInputX))

print(len(trainY))
print(len(trainInputY))

trainInputX = trainInputX [:,:,0:14]
testInputX = testInputX [:,:,0:14]
print(trainInputX.shape)

trained_models_rnn_400 = []
trained_models_rnn_400_history = []

for i in range(0,10):

    #giving it reproducibility
    seed = (i+1000)

    os.environ['PYTHONHASHseed']=str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    batch_size = 256

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
    
    trained_models_rnn_400.append(kr.Sequential())
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_400[i].add(kr.layers.SimpleRNN(14,return_sequences = True))
    trained_models_rnn_400[i].add(kr.layers.SimpleRNN(8))
    # rnn_model.add(kr.layers.SimpleRNN(5,input_shape=(trainInputX.shape[1],trainInputX.shape[2])))
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_400[i].add(kr.layers.Dense(1))
    trained_models_rnn_400[i].compile(loss='mean_squared_error',optimizer='adam')
    
    trained_models_rnn_400_history.append(trained_models_rnn_400[i].fit(trainInputX, trainInputY, epochs=2000, batch_size=batch_size, verbose = 1, validation_data=(testInputX,testInputY), callbacks=[callback]))
    
    trained_models_rnn_400[i].save('C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_400_batch_256_earlystop_valloss_'+str(i))

(11299, 400, 14)
(11299,)
(2532, 400, 14)
(2532,)
11699
11299
11699
11299
(11299, 400, 14)
Epoch 1/2000
45/45 [==============================] - 4s 68ms/step - loss: 0.0348 - val_loss: 0.0215
Epoch 2/2000
45/45 [==============================] - 3s 64ms/step - loss: 0.0052 - val_loss: 0.0101
Epoch 3/2000
45/45 [==============================] - 3s 67ms/step - loss: 0.0023 - val_loss: 0.0056
Epoch 4/2000
45/45 [==============================] - 3s 68ms/step - loss: 0.0014 - val_loss: 0.0038
Epoch 5/2000
45/45 [==============================] - 3s 66ms/step - loss: 9.9913e-04 - val_loss: 0.0031
Epoch 6/2000
45/45 [==============================] - 3s 69ms/step - loss: 8.0966e-04 - val_loss: 0.0030
Epoch 7/2000
45/45 [==============================] - 3s 70ms/step - loss: 7.0320e-04 - val_loss: 0.0027
Epoch 8/2000
45/45 [==============================] - 3s 70ms/step - loss: 6.2672e-04 - val_loss: 0.0026
Epoch 9/2000
45/45 [==============================] - 3s 69ms/step - loss: 5.6260e-04

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_400_batch_256_earlystop_valloss_0\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_400_batch_256_earlystop_valloss_0\assets


Epoch 1/2000
45/45 [==============================] - 4s 69ms/step - loss: 0.0236 - val_loss: 0.0128
Epoch 2/2000
45/45 [==============================] - 3s 64ms/step - loss: 0.0041 - val_loss: 0.0049
Epoch 3/2000
45/45 [==============================] - 3s 63ms/step - loss: 0.0025 - val_loss: 0.0032
Epoch 4/2000
45/45 [==============================] - 3s 64ms/step - loss: 0.0018 - val_loss: 0.0023
Epoch 5/2000
45/45 [==============================] - 3s 63ms/step - loss: 0.0014 - val_loss: 0.0017
Epoch 6/2000
45/45 [==============================] - 3s 64ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 7/2000
45/45 [==============================] - 3s 64ms/step - loss: 8.1613e-04 - val_loss: 0.0012
Epoch 8/2000
45/45 [==============================] - 3s 63ms/step - loss: 6.4161e-04 - val_loss: 0.0013
Epoch 9/2000
45/45 [==============================] - 3s 64ms/step - loss: 5.2226e-04 - val_loss: 0.0013
Epoch 10/2000
45/45 [==============================] - 3s 63ms/step - loss: 4.4

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_400_batch_256_earlystop_valloss_1\assets


...

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_400_batch_256_earlystop_valloss_8\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_400_batch_256_earlystop_valloss_8\assets


Epoch 1/2000
45/45 [==============================] - 4s 68ms/step - loss: 0.0166 - val_loss: 0.0413
Epoch 2/2000
45/45 [==============================] - 3s 64ms/step - loss: 0.0052 - val_loss: 0.0243
Epoch 3/2000
45/45 [==============================] - 3s 64ms/step - loss: 0.0029 - val_loss: 0.0140
Epoch 4/2000
45/45 [==============================] - 3s 66ms/step - loss: 0.0016 - val_loss: 0.0104
Epoch 5/2000
45/45 [==============================] - 3s 64ms/step - loss: 9.3105e-04 - val_loss: 0.0090
Epoch 6/2000
45/45 [==============================] - 3s 68ms/step - loss: 6.4691e-04 - val_loss: 0.0079
Epoch 7/2000
45/45 [==============================] - 3s 64ms/step - loss: 5.2169e-04 - val_loss: 0.0072
Epoch 8/2000
45/45 [==============================] - 3s 63ms/step - loss: 4.3841e-04 - val_loss: 0.0065
Epoch 9/2000
45/45 [==============================] - 3s 64ms/step - loss: 3.8488e-04 - val_loss: 0.0058
Epoch 10/2000
45/45 [==============================] - 3s 65ms/step - l

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_400_batch_256_earlystop_valloss_9\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_400_batch_256_earlystop_valloss_9\assets


In [16]:
rnn_memory = 800

trainInputX, trainInputY = sf.prepareData(trainX, trainY, rnn_memory)
testInputX, testInputY = sf.prepareData(testX, testY, rnn_memory)
print(trainInputX.shape)
print(trainInputY.shape)
print(testInputX.shape)
print(testInputY.shape)

#validacao tamanho apos tratamentos
print(len(trainX))
print(len(trainInputX))

print(len(trainY))
print(len(trainInputY))

trainInputX = trainInputX [:,:,0:14]
testInputX = testInputX [:,:,0:14]
print(trainInputX.shape)

trained_models_rnn_800 = []
trained_models_rnn_800_history = []

for i in range(0,5):

    #giving it reproducibility
    seed = (i+1000)

    os.environ['PYTHONHASHseed']=str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    batch_size = 256

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=24)
    
    trained_models_rnn_800.append(kr.Sequential())
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_800[i].add(kr.layers.SimpleRNN(14,return_sequences = True))
    trained_models_rnn_800[i].add(kr.layers.SimpleRNN(8))
    # rnn_model.add(kr.layers.SimpleRNN(5,input_shape=(trainInputX.shape[1],trainInputX.shape[2])))
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_800[i].add(kr.layers.Dense(1))
    trained_models_rnn_800[i].compile(loss='mean_squared_error',optimizer='adam')
    
    trained_models_rnn_800_history.append(trained_models_rnn_800[i].fit(trainInputX, trainInputY, epochs=2000, batch_size=batch_size, verbose = 1, validation_data=(testInputX,testInputY), callbacks=[callback]))
    
    trained_models_rnn_800[i].save('C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_800_batch_256_earlystop_valloss_'+str(i))

(10899, 800, 14)
(10899,)
(2132, 800, 14)
(2132,)
11699
10899
11699
10899
(10899, 800, 14)
Epoch 1/2000
43/43 [==============================] - 7s 132ms/step - loss: 0.0360 - val_loss: 0.0208
Epoch 2/2000
43/43 [==============================] - 6s 130ms/step - loss: 0.0055 - val_loss: 0.0094
Epoch 3/2000
43/43 [==============================] - 6s 129ms/step - loss: 0.0024 - val_loss: 0.0055
Epoch 4/2000
43/43 [==============================] - 6s 132ms/step - loss: 0.0014 - val_loss: 0.0038
Epoch 5/2000
43/43 [==============================] - 6s 134ms/step - loss: 0.0010 - val_loss: 0.0034
Epoch 6/2000
43/43 [==============================] - 6s 129ms/step - loss: 8.3004e-04 - val_loss: 0.0029
Epoch 7/2000
43/43 [==============================] - 6s 129ms/step - loss: 7.2946e-04 - val_loss: 0.0031
Epoch 8/2000
43/43 [==============================] - 5s 128ms/step - loss: 6.5268e-04 - val_loss: 0.0029
Epoch 9/2000
43/43 [==============================] - 6s 130ms/step - loss: 5.901

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_800_batch_256_earlystop_valloss_0\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_800_batch_256_earlystop_valloss_0\assets


Epoch 1/2000
43/43 [==============================] - 7s 131ms/step - loss: 0.0233 - val_loss: 0.0148
Epoch 2/2000
43/43 [==============================] - 5s 127ms/step - loss: 0.0042 - val_loss: 0.0061
Epoch 3/2000
43/43 [==============================] - 5s 126ms/step - loss: 0.0025 - val_loss: 0.0037
Epoch 4/2000
43/43 [==============================] - 5s 126ms/step - loss: 0.0018 - val_loss: 0.0023
Epoch 5/2000
43/43 [==============================] - 5s 126ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 6/2000
43/43 [==============================] - 5s 127ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 7/2000
43/43 [==============================] - 5s 128ms/step - loss: 8.0603e-04 - val_loss: 0.0011
Epoch 8/2000
43/43 [==============================] - 5s 126ms/step - loss: 6.3102e-04 - val_loss: 0.0012
Epoch 9/2000
43/43 [==============================] - 5s 127ms/step - loss: 5.1062e-04 - val_loss: 0.0013
Epoch 10/2000
43/43 [==============================] - 5s 126ms/step -

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_800_batch_256_earlystop_valloss_1\assets


...

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_800_batch_256_earlystop_valloss_3\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_800_batch_256_earlystop_valloss_3\assets


Epoch 1/2000
43/43 [==============================] - 7s 135ms/step - loss: 0.1321 - val_loss: 0.0462
Epoch 2/2000
43/43 [==============================] - 6s 131ms/step - loss: 0.0101 - val_loss: 0.0297
Epoch 3/2000
43/43 [==============================] - 6s 128ms/step - loss: 0.0071 - val_loss: 0.0295
Epoch 4/2000
43/43 [==============================] - 6s 134ms/step - loss: 0.0056 - val_loss: 0.0247
Epoch 5/2000
43/43 [==============================] - 6s 131ms/step - loss: 0.0044 - val_loss: 0.0233
Epoch 6/2000
43/43 [==============================] - 6s 129ms/step - loss: 0.0036 - val_loss: 0.0212
Epoch 7/2000
43/43 [==============================] - 6s 129ms/step - loss: 0.0030 - val_loss: 0.0188
Epoch 8/2000
43/43 [==============================] - 6s 129ms/step - loss: 0.0025 - val_loss: 0.0168
Epoch 9/2000
43/43 [==============================] - 6s 129ms/step - loss: 0.0021 - val_loss: 0.0144
Epoch 10/2000
43/43 [==============================] - 6s 129ms/step - loss: 0.001

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_800_batch_256_earlystop_valloss_4\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_800_batch_256_earlystop_valloss_4\assets


In [17]:
rnn_memory = 1000

trainInputX, trainInputY = sf.prepareData(trainX, trainY, rnn_memory)
testInputX, testInputY = sf.prepareData(testX, testY, rnn_memory)
print(trainInputX.shape)
print(trainInputY.shape)
print(testInputX.shape)
print(testInputY.shape)

#validacao tamanho apos tratamentos
print(len(trainX))
print(len(trainInputX))

print(len(trainY))
print(len(trainInputY))

trainInputX = trainInputX [:,:,0:14]
testInputX = testInputX [:,:,0:14]
print(trainInputX.shape)

trained_models_rnn_1000 = []
trained_models_rnn_1000_history = []

for i in range(0,5):

    #giving it reproducibility
    seed = (i+1000)

    os.environ['PYTHONHASHseed']=str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    batch_size = 256

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=24)
    
    trained_models_rnn_1000.append(kr.Sequential())
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_1000[i].add(kr.layers.SimpleRNN(14,return_sequences = True))
    trained_models_rnn_1000[i].add(kr.layers.SimpleRNN(8))
    # rnn_model.add(kr.layers.SimpleRNN(5,input_shape=(trainInputX.shape[1],trainInputX.shape[2])))
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_1000[i].add(kr.layers.Dense(1))
    trained_models_rnn_1000[i].compile(loss='mean_squared_error',optimizer='adam')
    
    trained_models_rnn_1000_history.append(trained_models_rnn_1000[i].fit(trainInputX, trainInputY, epochs=2000, batch_size=batch_size, verbose = 1, validation_data=(testInputX,testInputY), callbacks=[callback]))
    
    trained_models_rnn_1000[i].save('C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_1000_batch_256_earlystop_valloss_'+str(i))

(10699, 1000, 14)
(10699,)
(1932, 1000, 14)
(1932,)
11699
10699
11699
10699
(10699, 1000, 14)
Epoch 1/2000
42/42 [==============================] - 9s 177ms/step - loss: 0.0368 - val_loss: 0.0201
Epoch 2/2000
42/42 [==============================] - 7s 171ms/step - loss: 0.0058 - val_loss: 0.0113
Epoch 3/2000
42/42 [==============================] - 7s 173ms/step - loss: 0.0026 - val_loss: 0.0068
Epoch 4/2000
42/42 [==============================] - 7s 172ms/step - loss: 0.0015 - val_loss: 0.0047
Epoch 5/2000
42/42 [==============================] - 7s 173ms/step - loss: 0.0011 - val_loss: 0.0039
Epoch 6/2000
42/42 [==============================] - 7s 171ms/step - loss: 8.7914e-04 - val_loss: 0.0034
Epoch 7/2000
42/42 [==============================] - 7s 172ms/step - loss: 7.6229e-04 - val_loss: 0.0030
Epoch 8/2000
42/42 [==============================] - 7s 173ms/step - loss: 6.8043e-04 - val_loss: 0.0030
Epoch 9/2000
42/42 [==============================] - 7s 171ms/step - loss: 6.

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_1000_batch_256_earlystop_valloss_0\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_1000_batch_256_earlystop_valloss_0\assets


Epoch 1/2000
42/42 [==============================] - 9s 178ms/step - loss: 0.0233 - val_loss: 0.0158
Epoch 2/2000
42/42 [==============================] - 7s 172ms/step - loss: 0.0042 - val_loss: 0.0072
Epoch 3/2000
42/42 [==============================] - 7s 171ms/step - loss: 0.0026 - val_loss: 0.0046
Epoch 4/2000
42/42 [==============================] - 7s 168ms/step - loss: 0.0019 - val_loss: 0.0029
Epoch 5/2000
42/42 [==============================] - 7s 170ms/step - loss: 0.0014 - val_loss: 0.0018
Epoch 6/2000
42/42 [==============================] - 7s 169ms/step - loss: 0.0011 - val_loss: 0.0013
Epoch 7/2000
42/42 [==============================] - 7s 169ms/step - loss: 8.4975e-04 - val_loss: 0.0011
Epoch 8/2000
42/42 [==============================] - 7s 171ms/step - loss: 6.6462e-04 - val_loss: 0.0011
Epoch 9/2000
42/42 [==============================] - 7s 170ms/step - loss: 5.3023e-04 - val_loss: 0.0013
Epoch 10/2000
42/42 [==============================] - 7s 171ms/step -

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_1000_batch_256_earlystop_valloss_1\assets


...

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_1000_batch_256_earlystop_valloss_3\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_1000_batch_256_earlystop_valloss_3\assets


Epoch 1/2000
42/42 [==============================] - 8s 175ms/step - loss: 0.1338 - val_loss: 0.0470
Epoch 2/2000
42/42 [==============================] - 7s 170ms/step - loss: 0.0103 - val_loss: 0.0319
Epoch 3/2000
42/42 [==============================] - 7s 171ms/step - loss: 0.0072 - val_loss: 0.0282
Epoch 4/2000
42/42 [==============================] - 7s 169ms/step - loss: 0.0057 - val_loss: 0.0287
Epoch 5/2000
42/42 [==============================] - 7s 170ms/step - loss: 0.0045 - val_loss: 0.0233
Epoch 6/2000
42/42 [==============================] - 7s 169ms/step - loss: 0.0037 - val_loss: 0.0218
Epoch 7/2000
42/42 [==============================] - 7s 171ms/step - loss: 0.0031 - val_loss: 0.0218
Epoch 8/2000
42/42 [==============================] - 7s 170ms/step - loss: 0.0026 - val_loss: 0.0172
Epoch 9/2000
42/42 [==============================] - 7s 171ms/step - loss: 0.0022 - val_loss: 0.0151
Epoch 10/2000
42/42 [==============================] - 7s 171ms/step - loss: 0.001

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_1000_batch_256_earlystop_valloss_4\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting1ahead_param_14_ts_1000_batch_256_earlystop_valloss_4\assets
